This notebook takes a script string or file and generates TTS audio from it. The script is chunked by section, using markdown headers as delimiters. Each chunk of text is generated into audio using ElevenLabs.

Section titles are saved as separate chunks then the content to allow for more natural pauses between sections.

In [ ]:
import os
from src.ai import elevenlabs
from src.utils import saveB64Audio, chunkTextForTTS, create_project_folder
from src.enums import ElevenLabsTTSModel

# if not specified, will use default model from config/config.yaml
tts_model = ElevenLabsTTSModel.Turbo_v25.value

script = """"""
script_location = "input.txt"
specific_chunk = -1

def create_audio(script, audio_name='audio.mp3', project_folder=None, model = None):
    if project_folder is None:
        project_folder = create_project_folder()

    audio = elevenlabs.getSpeechB64(script, model)
    audio_path = os.path.join(project_folder, audio_name)
    saveB64Audio(audio, audio_path)

    print(f"Audio {audio_name} created successfully in {project_folder}")
    return project_folder, audio_path

input_exists = os.path.exists(script_location)
if not script and not input_exists:
    raise SystemExit("Please provide a script or a script location")

if not script and script_location:
    with open(script_location, 'r') as f:
        script = f.read()

chunks = chunkTextForTTS(script)
project_folder = 'project_20241014_141200'

# print chunks
# for i, chunk in enumerate(chunks):
#     print(f"{i}: {chunk['type']} - {chunk['content']}")
# raise SystemExit()

if specific_chunk >= 0:
    create_audio(chunks[specific_chunk]['content'], f'{specific_chunk}_audio.mp3', project_folder)
else:
    skipped = 0
    for i, chunk in enumerate(chunks):
        chunk_type = chunk['type']
        audio_name = f'{i}_{chunk_type}.mp3'
        audio_exists = os.path.exists(os.path.join(project_folder, audio_name))

        if audio_exists:
            skipped += 1
            continue

        content = chunk['content']
        create_audio(content, audio_name, project_folder, tts_model)

    print(f"Skipped generating {skipped} audio files")

In [8]:
# elevenlabs.getModels()